# 多头注意力

# 1 导入相关库

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

### 1.1 reshape与view是怎么进行的?  
先将原张量从shape的最后一个维度排成一行,然后再按照reshape与view中的参数来排,也是一行一行排的.flatten也是这样做的.   
<img src = "reshape与view是怎么操作的.png">  
### 1.2 transpose与permute是怎么进行的?
而transpose与permute是变换维度,其中transpose是一次只能变两个维度,而permute是一次可以变多个维度.我们以transpose为例说一下维度是如何变换的.   
如transpose(1, 2),这是交换维度1与维度2,具体交换维度方式为,原矩阵的第(i, j, k, m)位置的值放到新矩阵的第(i, k, j, m)位置.

## 2 self-attention
$1$ signal_head_self-attention中每个向量的输入与输出的长度是一样的，所以在单头的self-attention中v的长度就是b的长度，因为b是由v乘注意力分数，而注意力分数是一个标量。一般来说qkv的长度是一样的,也可以设为不一样.     
$2$ multi_head_self-attention中每个向量的输入与输出的长度是一样的，而每个输入的n个头是在列上concate的，得到b1，然后将每个输入得到的b1按行concate就得到了总的b，在对b进行乘一个w矩阵就得到最后的B。如下图所示。
<center>
    <img src = "多头注意力.png">
<center>

In [2]:
class Self_Attention(nn.Module):
    """
        注意力函数,可以通过参数指定是多头还是单头,注意力评分函数用的是Dot-Product.

        Parameters:
            toekn_size:每个token向量的长度.
            qk_size:每个q与k向量的长度.
            v_size:每个v向量的长度.当已有一个头时,qkv三向量的长度是一样的.多头时一般qkv长度也是一样的.
                    只要保证最后输出token的长度与输入token的长度一样的就可以.
            head_num:有几个头.

        Returns:
    """
    def __init__(self, token_size, qk_size, v_size, head_num):
        super().__init__()

        self.token_size = token_size
        self.qk_size = qk_size
        self.v_size = v_size
        self.head_num = head_num

        # 生成qkv的全连接层。
        self.W_q = nn.Linear(token_size, qk_size * head_num)
        self.W_k = nn.Linear(token_size, qk_size * head_num)
        self.W_v = nn.Linear(token_size, v_size * head_num)
        self.scale = 1 / torch.sqrt(torch.tensor(qk_size))

        # 如果是多头的话，最后还有一个可学习参数矩阵。
        self.W = nn.Linear(v_size * head_num, token_size)
    

    """
        注意力函数,可以通过参数指定是多头还是单头,注意力评分函数用的是Dot-Product.

        Parameters:
            x:形状为(batch, token数量, 每个token的长度)

        Returns:
    """
    def forward(self, x):
        batch, token_num, token_size = x.shape
        assert self.token_size == token_size, "判断类参数token长度与输入变量token长度不一样"

        # 变换维度为(batch数, token数, head头数, qkv向量的长度)
        # 又transpose交换维度后变为(batch数, head头数, token数, qkv向量长度)
        q = self.W_q(x).contiguous().view(batch, token_num, self.head_num, self.qk_size).transpose(1, 2)
        k = self.W_k(x).contiguous().view(batch, token_num, self.head_num, self.qk_size).transpose(1, 2)
        v = self.W_v(x).contiguous().view(batch, token_num, self.head_num, self.v_size).transpose(1, 2)

        # 得到相似度得分
        score = torch.matmul(q, k.transpose(2, 3)) * self.scale
        score = torch.softmax(score, dim = -1) # (batch, head头数, token数, token数)

        # 得到多头的b,维度为(batch, head头数, token数, v向量的长度)
        # transpose(1, 2)之后维度为(batch, token数, head头数, v向量的长度)
        b_t_h = torch.matmul(score, v).transpose(1, 2)
        
        # 将维度变为(bathch, token数, head头数 * v向量长度)
        b_concate = b_t_h.contiguous().view(batch, token_num, self.head_num * self.v_size)

        # 输出的维度为(batch, token数, b向量的长度)
        b = self.W(b_concate)

        return b

In [3]:
x = torch.rand((2, 128, 100))
multi_head_att = Self_Attention(x.shape[2], 2, 2, 12)
b = multi_head_att(x)
b

tensor([[[-0.2756, -0.1439, -0.0618,  ...,  0.1857, -0.0534,  0.5218],
         [-0.2755, -0.1445, -0.0611,  ...,  0.1868, -0.0537,  0.5219],
         [-0.2750, -0.1441, -0.0622,  ...,  0.1864, -0.0537,  0.5213],
         ...,
         [-0.2745, -0.1438, -0.0622,  ...,  0.1866, -0.0539,  0.5214],
         [-0.2750, -0.1440, -0.0618,  ...,  0.1857, -0.0536,  0.5214],
         [-0.2750, -0.1443, -0.0617,  ...,  0.1866, -0.0535,  0.5214]],

        [[-0.2555, -0.1716, -0.0731,  ...,  0.2135, -0.0547,  0.5165],
         [-0.2551, -0.1718, -0.0727,  ...,  0.2143, -0.0549,  0.5170],
         [-0.2545, -0.1710, -0.0725,  ...,  0.2141, -0.0553,  0.5170],
         ...,
         [-0.2559, -0.1721, -0.0730,  ...,  0.2139, -0.0545,  0.5163],
         [-0.2551, -0.1717, -0.0722,  ...,  0.2144, -0.0552,  0.5171],
         [-0.2547, -0.1716, -0.0724,  ...,  0.2134, -0.0549,  0.5165]]],
       grad_fn=<ViewBackward0>)